# Imports

In [ ]:
import copy
import cv2
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import ImageColor
from skimage import exposure
from skimage.measure import label, regionprops
from skimage.morphology import area_opening, disk, square
import sys

# Helper Functions

In [ ]:
def plot_images(image_list, title_list=[], is_barchart=False, grid='off', size_per_image=10):
    
    if len(image_list) > 20: # this assumes fewer than 20 images will ever be displayed
        image_list = [image_list]

    # ----------------------------------------
    
    if is_barchart:        
        fig, axes = plt.subplots(1, ncols=len(image_list), figsize=(size_per_image*len(image_list), size_per_image*1))
        if len(image_list) == 1:
            axes = [axes]
        
        for i, ax in enumerate(axes):
            ax.bar([str(i) for i in np.arange(len(image_list[i]))], image_list[i], align='center') # TODO set bar chart color to match score from heatmap, ylabels need to be consistent
            ax.set_xlabel('Score')
            ax.set_ylabel('%')
            if len(title_list) > 0:
                ax.set_title(title_list[i])
            ax.axis(grid)
    
    # ----------------------------------------
    
    else:
        fig, axes = plt.subplots(nrows=1, ncols=len(image_list), figsize=(size_per_image*len(image_list), size_per_image*1))
        if len(image_list) == 1:
            axes = [axes]

        for i, ax in enumerate(axes):
            ax.imshow(cv2.cvtColor(image_list[i], cv2.COLOR_BGR2RGB))
            if len(title_list) > 0:
                ax.set_title(title_list[i])
            ax.axis(grid)

In [ ]:
# BOUNDING BOX

def create_bounding_boxes(mission_numbers, masks, output_images, num_bbox_iterations=1, bbox_color=(255, 0, 0), debug_images=False):
    temp = copy.deepcopy(masks)
    temp_bounding_box_masks = []

    num_bbox = []
    
    for k in range(0, num_bbox_iterations):
        bounding_box_masks = []

        for i in range(0, len(temp)):
            label_ = label(temp[i])
            props_ = regionprops(label_)
            bounding_mask = np.zeros((temp[i].shape)).astype(np.uint8)

            if k == num_bbox_iterations-1:
                num_bbox.append(len(props_))

            factor = 0
            for prop in props_:
                if k == num_bbox_iterations-1:
                    cv2.rectangle(bounding_mask, (prop.bbox[1], prop.bbox[0]), (prop.bbox[3], prop.bbox[2]), bbox_color, 2)
                else:
                    cv2.rectangle(bounding_mask, (prop.bbox[1]-factor, prop.bbox[0]-factor), (prop.bbox[3]+2*factor, prop.bbox[2]+2*factor), bbox_color, -1)

            bounding_box_masks.append(bounding_mask)

        temp_bounding_box_masks.append(copy.deepcopy(bounding_box_masks))
        temp = copy.deepcopy(bounding_box_masks)
    
    if debug_images:
        for k in range(0, num_bbox_iterations):
            plot_images(temp_bounding_box_masks[k], ['bounding_box_masks ' + str(mission_number) for mission_number in mission_numbers])
    
    # ----------------------------------------
    
    results = []
    for i in range(0, len(mission_numbers)):
        result = copy.deepcopy(output_images[i])
        result[temp_bounding_box_masks[num_bbox_iterations-1][i].astype(bool), :] = bbox_color
        results.append(result)
    
    return bounding_box_masks, num_bbox, None

In [ ]:
# COLOR GRADIENT

def color_fader(cs, mix=0):
    if len(cs) == 2:
        c1 = np.array(mpl.colors.to_rgb(cs[0]))
        c2 = np.array(mpl.colors.to_rgb(cs[1]))
        return mpl.colors.to_hex((1-mix)*c1 + mix*c2)
    elif len(cs) == 3:
        c1 = np.array(mpl.colors.to_rgb(cs[0]))
        c2 = np.array(mpl.colors.to_rgb(cs[1]))
        c3 = np.array(mpl.colors.to_rgb(cs[2]))
        if mix <= .5:
            mix *= 2 # create 0-1 range
            return mpl.colors.to_hex((1-mix)*c1 + mix*c2)
        else:
            mix -= .5 # create 0-1 range
            mix *= 2
            return mpl.colors.to_hex((1-mix)*c2 + mix*c3)

# ----------------------------------------
        
def create_color_buckets(colors=['#FF0000', '#00FF00'], num_buckets=10, debug_images=False): #RGB
    color_buckets = []

    if False:
        fig, ax = plt.subplots(figsize=(4, 2))
    
    for x in range(0, num_buckets):
        X = color_fader(colors, x/(num_buckets-1))
        
        if False:
            ax.axvline(x, color=X, linewidth=4)
        
        color_buckets.append(ImageColor.getcolor(X, "RGB")[::-1]) # convert RGB to BGR

    if False:
        plt.axis('off')
        plt.show()

    return color_buckets

# Shadow Removal

In [ ]:
def create_shadow_masks(mission_numbers, mission_images_hsv, debug_images=False):
        
    shadow_masks = []
    smoothed_shadow_masks = []

    for i in range(0, len(mission_numbers)):
        lower_shadow = np.array([69, 20, 1])
        upper_shadow = np.array([129, 102, 34])
        mask = cv2.inRange(mission_images_hsv[i].copy(), lower_shadow, upper_shadow)
        shadow_masks.append(mask)

        mask = copy.deepcopy(mask)
        mask = area_opening(mask, area_threshold=50)
        mask = cv2.dilate(mask, disk(radius=5), iterations=1)
        smoothed_shadow_masks.append(mask)

    # ----------------------------------------
    
    complete_shadow_mask = np.zeros((smoothed_shadow_masks[0].shape)).astype(int)
    for i in range(0, len(shadow_masks)):
        complete_shadow_mask = cv2.bitwise_or(complete_shadow_mask, smoothed_shadow_masks[i].astype(int))

    complete_shadow_mask = complete_shadow_mask.astype(np.uint8)
    
    # ----------------------------------------
    
    if False:
        plot_images(shadow_masks, ['shadow_masks ' + str(mission_number) for mission_number in mission_numbers])
        plot_images(smoothed_shadow_masks, ['smoothed_shadow_masks ' + str(mission_number) for mission_number in mission_numbers])
        plot_images([complete_shadow_mask], ['complete_shadow_mask'])
        
    return shadow_masks, smoothed_shadow_masks, complete_shadow_mask

# ----------------------------------------

def apply_shadow_masks(mission_numbers, histogram_mission_images_hsv, complete_shadow_mask, debug_images=False):
    modified_mission_images_hsv = []

    for i in range(0, len(mission_numbers)):
        modified_hsv = copy.deepcopy(histogram_mission_images_hsv[i])

        try:
            modified_hsv[complete_shadow_mask.astype(bool), :] = 0 # set to black
        except ValueError:
            print('not modified')

        modified_mission_images_hsv.append(modified_hsv)
        
    if False:
        plot_images(modified_mission_images_hsv, ['modified_mission_images_hsv ' + str(mission_number) for mission_number in mission_numbers])
        
    return modified_mission_images_hsv

In [ ]:
def run_shadows(mission_numbers, mission_images_hsv, histogram_mission_images_hsv, debug_images=False):
    shadow_masks, smoothed_shadow_masks, complete_shadow_mask = create_shadow_masks(mission_numbers, mission_images_hsv, debug_images)
    modified_mission_images_hsv = apply_shadow_masks(mission_numbers, histogram_mission_images_hsv, complete_shadow_mask, debug_images)
    return modified_mission_images_hsv

# Anomaly Detection

In [ ]:
def create_pixel_transformation_function(debug_images=False):
    box_w = 100
    box_y = 123

    x_lim = np.array([0, int((255-box_w)/2), 255 - int((255-box_w)/2), 255])
    y_lim = np.array([0,  box_y, box_y, 255])
    z = np.polyfit(x_lim, y_lim, 5)

    nonlinear_poly = np.poly1d(z)

    xp = np.linspace(0, 255, 256)
    yp = nonlinear_poly(xp)

    xp_adj = np.linspace(0, 255, 256)
    yp_adj = nonlinear_poly(xp_adj)
    mask = [num >= x_lim[1] and num <= x_lim[2] for num in xp_adj]
    yp_adj[mask] = box_y
    
    if False:
        plt.plot(x_lim, y_lim, 'k', xp, yp, 'r', xp, yp_adj, 'b')
    
    return nonlinear_poly, x_lim, y_lim, box_y

In [ ]:
def differencing(img1, img2):
#     img1 = copy.deepcopy(img1[:, :, 0]) # using HSV channel
#     img2 = copy.deepcopy(img2[:, :, 0])

#     img1 = img1.astype(np.int16) # SUPER IMPORTANT - convert to int16 (or higher but necessary) to so that "result" array can preserve greater than +-255
#     img2 = img2.astype(np.int16)
    
#     result = img1 - img2

#     result += 255 # max difference is 255 or -255, add 255 to bring everything positive

#     result = (result-np.min(result))*(255/(np.max(result) - np.min(result))) # linear 0-255 scaling

#     xp_adj = copy.deepcopy(result)
#     yp_adj = nonlinear_poly(xp_adj) # nonlinear 0-255 scaling

#     mask = cv2.inRange(xp_adj.copy(), np.array([x_lim[1]]).astype(np.uint8), np.array([x_lim[2]]).astype(np.uint8)).astype(bool)

#     result[mask] = box_y
        
#     result = result.astype(np.uint8) # convert back to uint8 to be displayed
    
    
    img1 = copy.deepcopy(img1[:, :, 1])
    img2 = copy.deepcopy(img2[:, :, 1])

    img1 = img1.astype(np.int16)
    img2 = img2.astype(np.int16)
    
    result = img1 - img2

    result += 255 

    result = (result-np.min(result))*(255/(np.max(result) - np.min(result)))
    
    result[result < 75] = 0 # darken
    result[result > 175] = 255 # lighten
        
    result = result.astype(np.uint8) # convert back to uint8 to be displayed
    
    return result

In [ ]:
# using modified_mission_images_hsv[0]

def compute_anomaly_masks(mission_numbers, modified_mission_images_hsv, debug_images=False):
    
#     nonlinear_poly, x_lim, y_lim, box_y = create_pixel_transformation_function(debug_images)

    # ----------------------------------------
    
    adjacent_regions_change_hsv = []

    for i in range(0, len(mission_numbers)-1):
        adjacent_regions_change_hsv.append(differencing(modified_mission_images_hsv[i], modified_mission_images_hsv[i+1]))

    # ----------------------------------------
    
    masks_initial = []
    masks_2 = []
    masks_final = []

    for i in range(0, len(adjacent_regions_change_hsv)):
        mask_l = adjacent_regions_change_hsv[i] == 0
        mask_u = adjacent_regions_change_hsv[i] == 255
        mask = (np.bitwise_or(mask_l, mask_u)*255).astype(np.uint8)
        masks_initial.append(mask)
        
        mask = cv2.erode(masks_initial[i], disk(radius=1), iterations=2).astype(np.uint8)
        masks_2.append(mask)
        
        mask = cv2.dilate(masks_2[i], disk(radius=5), iterations=1).astype(np.uint8)
        masks_final.append(mask)

    # ----------------------------------------
    
    if debug_images:
        plot_images(adjacent_regions_change_hsv, ['adjacent_regions_change_hsv ' + str(mission_numbers[i]) + '-' + str(mission_numbers[i+1]) for i in range(0, len(mission_numbers)-1)])
#         plot_images(masks_initial, ['masks_initial ' + str(mission_number) for mission_number in mission_numbers])
#         plot_images(masks_2, ['masks_2 ' + str(mission_number) for mission_number in mission_numbers])        
        plot_images(masks_final, ['masks_final ' + str(mission_numbers[i]) + '-' + str(mission_numbers[i+1]) for i in range(0, len(mission_numbers)-1)])
        
    return adjacent_regions_change_hsv, masks_final

In [ ]:
def anomaly_detection_1(mission_numbers, mission_images_hsv, histogram_mission_images_bgr, histogram_mission_images_hsv, num_bbox_iterations=1, bbox_color=(255, 0, 0), debug_images=False):
    print('Anomaly Detection 1')    
    print('Step 1 - detecting shadows')
    modified_mission_images_hsv = run_shadows(mission_numbers, mission_images_hsv, histogram_mission_images_hsv, debug_images)
    print('Step 2 - detecting anomalies')
    adjacent_regions_change_hsv, anomaly_masks = compute_anomaly_masks(mission_numbers, modified_mission_images_hsv, debug_images)
    print('Step 3 - creating bounding boxes')
    bbox_masks, num_bboxs, superimposed_images = create_bounding_boxes(mission_numbers, anomaly_masks, histogram_mission_images_bgr, num_bbox_iterations, bbox_color, debug_images)
    print('Done')
#     return bbox_masks, num_bboxs, superimposed_images
    return None

# Usage

In [ ]:
# bbox_masks, num_bboxs, superimposed_images = anomaly_detection_1(mission_numbers, mission_images_hsv, histogram_mission_images_bgr, histogram_mission_images_hsv, bbox_color=BBOX_COLOR, debug_images=False)

In [ ]:
# print(num_bboxs)
# plot_images(bbox_masks, ['bbox_masks ' + str(mission_number) for mission_number in mission_numbers])
# plot_images(superimposed_images, ['superimposed_images ' + str(mission_number) for mission_number in mission_numbers])

### as final output return bbox_masks. these will be superimposed on other bbox_masks, bbox x, y, w, h